In [2]:
# macfrom IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

# clear_output()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.6 MB/s 
     |████████████████████████████████| 3.3 MB 49.3 MB/s 
     |████████████████████████████████| 880 kB 56.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a2f445188e68ca6b6a481e538a418ecef3d07f857bd0933469bfcc0e4de70679
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 15.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 28.9 MB/s 


In [3]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

1.0.2
2.8.2
4.8.2
0.17.1


In [8]:
import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

In [9]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [6]:
from google.colab import files
file_uploaded = files.upload()

In [10]:
df_train = pd.read_csv("unsmile_train_new.csv")
df_test = pd.read_csv("unsmile_valid_new.csv")

In [11]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from konlpy.tag import Okt

In [13]:
def make_corpus(sentence):
    okt = Okt()
    raw_pos_tagged = okt.pos(sentence, norm=True, stem=True)
    word_cleaned = []

    for word in raw_pos_tagged:
        if word[1] in ['Noun', 'Adjective', 'Verb']:
            word_cleaned.append(word[0])
    return ' '.join(word_cleaned)

In [14]:
df_train.head()

,문장,target
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,1
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1


In [15]:
df_test.head()

,문장,target
0,ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...,1
1,여자들은 취미가 애낳는건가.. 취미를 좀 가져라,1
2,개슬람녀 다 필요없고 니 엄마만 있으면 된다,1
3,조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...,1
4,바지 내리다 한남들 와꾸 보고 올려뿟노,1


In [17]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
# clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

fatal: destination path 'KoBERT-Transformers' already exists and is not an empty directory.
mv: cannot stat 'KoBERT-Transformers/kobert_transformers/tokenization_kobert.py': No such file or directory


In [18]:
df_train['문장'].str.len().sort_values(ascending=False)[:5]

19616    181
22062    155
10374    155
12163    150
5478     150
Name: 문장, dtype: int64

In [19]:
len(tokenizer.tokenize(df_train['문장'][19616]))

176

In [20]:
X_train = df_train['문장']
X_test = df_test['문장']
y_train = df_train['target']
y_test = df_test['target']

In [21]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (22532,)
X_test shape:  (5604,)
y_train shape:  (22532,)
y_test shape:  (5604,)


In [22]:
SEQ_LEN = 300

token_ids = []
token_segments = []
token_masks = []

train_labels = []

for idx in tqdm(range(len(X_train))):

  # 'Sentence'칼럼에서 0번째 행~ 꺼내오기
  train_sentence = X_train.iloc[idx]
  print(train_sentence) 

  # 특수문자 제거
  cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", train_sentence)
  print(cleaned_sentence)

  # 토큰화 + 토큰별 시퀀스번호 부여 + 패딩
  encoded_dict = tokenizer.encode_plus(text = cleaned_sentence,
                                       padding='max_length',
                                       truncation=True,
                                       max_length=SEQ_LEN
                                       )
  
  #'input_ids', 'token_type_ids', 'attention_mask'
  token_ids.append(encoded_dict['input_ids'])
  token_masks.append(encoded_dict['attention_mask'])
  token_segments.append(encoded_dict['token_type_ids'])

  train_labels.append(y_train.iloc[idx])


train_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
train_labels = np.array(train_labels)

  1%|          | 214/22532 [00:00<00:20, 1086.24it/s]

일안하는 시간은 쉬고싶어서 그런게 아닐까
일안하는 시간은 쉬고싶어서 그런게 아닐까
아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. 10 16까지 IQ 떨어지고 출산 위험은 400% 다.
아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다 무슬림 50퍼 근친이다 10 16까지 IQ 떨어지고 출산 위험은 400 다
루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o  doin 진짜 띵곡임 꼭 들어보셈"
루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o  doin 진짜 띵곡임 꼭 들어보셈
홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽에서 고소 가능하냐?
홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽에서 고소 가능하냐
아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런다! 이렇게 2개로 싸우냐.. 내가 볼 땐 둘다 나쁜 말 그 이상도 이하도 아닌데
아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다 여자들은 더 심하게 그런다 이렇게 2개로 싸우냐 내가 볼 땐 둘다 나쁜 말 그 이상도 이하도 아닌데
고향가서 피방가면 동네 부럴 친구들이랑은 뭐 거르는 거 없이 이야기하니까 막 말하게 되더라 당연히 키보드를 치거나 그러지는 않는데 말하는게 많이 거칠어지긴 해 반성해야겠네
고향가서 피방가면 동네 부럴 친구들이랑은 뭐 거르는 거 없이 이야기하니까 막 말하게 되더라 당연히 키보드를 치거나 그러지는 않는데 말하는게 많이 거칠어지긴 해 반성해야겠네
예수 십새끼 개새끼 창녀아들 애비실종 가정교육 못받은 무뇌충 허언증 정신병자 사기꾼 망상 근본없는 병신 장애인 짱개 흑인 이슬람 전라도랑 동급
예수 십새끼 개새끼 창녀아들 애비실종 가정교육 못받은 무뇌충 허언증 정신병자 사기꾼 망상 근본없는 병신 장애인 짱개 흑인 이슬람 전라도랑 동급
나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임
나이쳐먹고 피시방가는 놈들은

  1%|▏         | 323/22532 [00:00<00:25, 869.18it/s] 

신천지들이 계획이 다 있구나 동성애자들이랑 연합해서 이런식으로 몰구 가보려구? ㅋㅋ
신천지들이 계획이 다 있구나 동성애자들이랑 연합해서 이런식으로 몰구 가보려구 ㅋㅋ
너그들 이나잘하세요. ...
너그들 이나잘하세요 
트랜스가본 2020년을 알아야되나? 쓸 기사가 그리도 없어? 아~~~~경향이구나...
트랜스가본 2020년을 알아야되나 쓸 기사가 그리도 없어 아경향이구나
답을알려줌 공수처는 중국 북한에만 있다 저거보면 딱 알겠지???ㅋㅋㅋㅋ
답을알려줌 공수처는 중국 북한에만 있다 저거보면 딱 알겠지ㅋㅋㅋㅋ
깨닫는거지 공부하는게 아니노
깨닫는거지 공부하는게 아니노
씨발 이나라 땅값은 알고 말하는거냐. 나도 난민신청하고 집받고 싶다
씨발 이나라 땅값은 알고 말하는거냐 나도 난민신청하고 집받고 싶다
코로나19 팬데믹 황폐화된 세상을 예수그리스도의 생명의 복음으로 녹색화하자
코로나19 팬데믹 황폐화된 세상을 예수그리스도의 생명의 복음으로 녹색화하자
순수한 풀스윙
순수한 풀스윙
남유럽 중유럽 동유럽 다 난민 받아주는데죽자고 서유럽으로만 가는거 보면 도둑놈 새끼들임
남유럽 중유럽 동유럽 다 난민 받아주는데죽자고 서유럽으로만 가는거 보면 도둑놈 새끼들임
보아하니 3대종교를 다 건드린 듯... 지들이 IS인줄 아나 봄..
보아하니 3대종교를 다 건드린 듯 지들이 IS인줄 아나 봄
게이도 좆같고 레즈도 좆같고 동성애 씨발년놈들은 옛날처럼 아우슈비츠 가스실에 싸그리 모아놓고 독가스 살포해서애미애비 보는앞에서 고통스럽고 잔인하게 죽여야된다
게이도 좆같고 레즈도 좆같고 동성애 씨발년놈들은 옛날처럼 아우슈비츠 가스실에 싸그리 모아놓고 독가스 살포해서애미애비 보는앞에서 고통스럽고 잔인하게 죽여야된다
무조건 이해하라고 하는것도 폭력이지
무조건 이해하라고 하는것도 폭력이지
노친네들 잡아죽이는게애국
노친네들 잡아죽이는게애국
젠신병은 정신병이노.
젠신병은 정신병이노
다문화걱정하지마 kkk단결성해서 쓸어버리면되
다문화걱정하지마 kkk단결성해서 쓸어버리면되
스시남 패고다니는거 개웃챙이노ㅋㅋㅋㅋ
스

  2%|▏         | 510/22532 [00:00<00:25, 867.76it/s]

...


부정부패랑 범죄자들만 제대로 잡히면 필리핀은 정말 좋은국가가 될듯
어떤 나라랑 다르다
어떤 나라랑 다르다
속이 없는 바나나껍질
속이 없는 바나나껍질
마리텔이 망한 이유를 알것같네ㅋㅋ
마리텔이 망한 이유를 알것같네ㅋㅋ
백야 김좌진 청산리전투
백야 김좌진 청산리전투
댓글이 없 어 지 네.....
댓글이 없 어 지 네
촛불들고 나가지 그래
촛불들고 나가지 그래
뭘 교환이야 환불하고 다른거사
뭘 교환이야 환불하고 다른거사
사기꾼 맞다에..한표
사기꾼 맞다에한표
이제 각자 자기길 가는거지.
이제 각자 자기길 가는거지
요즘 날씨에서 미세먼지 얘기는 안한기로 약속했나요??
요즘 날씨에서 미세먼지 얘기는 안한기로 약속했나요
시대의 흐름을 보니 트럼프 당선이 확실시 되는듯
시대의 흐름을 보니 트럼프 당선이 확실시 되는듯
옵션있어서 85억은 훌쩍 넘길듯ㅋㅋㅋㅋㅋㅋ
옵션있어서 85억은 훌쩍 넘길듯ㅋㅋㅋㅋㅋㅋ
눈알을 빼는 범죄가 생길까
눈알을 빼는 범죄가 생길까
뒷북을 울려라 둥둥둥~
뒷북을 울려라 둥둥둥
정청파 식구인가 보네...
정청파 식구인가 보네
남편 월수입은? 부인이 다버는거 아녀?
남편 월수입은 부인이 다버는거 아녀
냉철하고 자비없는 삼성의 역공을 기대합니다~
냉철하고 자비없는 삼성의 역공을 기대합니다
잊을만하면 어글잡는 기사 뜨네
잊을만하면 어글잡는 기사 뜨네
자원나끼고 전자책이 도옹야 할ㄹ때가 온듯
자원나끼고 전자책이 도옹야 할ㄹ때가 온듯
좋아 이제 문유를 재현하면 되는건가?
좋아 이제 문유를 재현하면 되는건가
인류의 발전은 모두 백인에게 비롯된거다
인류의 발전은 모두 백인에게 비롯된거다
사진 내리면서 보는데 홍종현 사자인줄 알았네...
사진 내리면서 보는데 홍종현 사자인줄 알았네
내가 그런 듯 한데.....
내가 그런 듯 한데
우리 시어머니평범한 50댄데 자꾸 이런글 퍼다날림...
우리 시어머니평범한 50댄데 자꾸 이런글 퍼다날림
동전던지기로 날씨 예보하는게 더 정확할듯
동전던지기로 날씨 예보하는게 더 정확할듯
추신수가 3타석에서 출루가 아니라 3홈런을 쳤어도 이

100%|█████████▉| 22442/22532 [00:33<00:00, 960.91it/s]


아이가 성인이 될때까지 이사는 안가는게 좋다
역으로 소형 emp탄을 개발하면 되지
역으로 소형 emp탄을 개발하면 되지
한화 이글스 피에인가 예전 2014년 함께 울고 웃었던 그 선수도 있잖슴!!!
한화 이글스 피에인가 예전 2014년 함께 울고 웃었던 그 선수도 있잖슴
딴나라 사람들인가??
딴나라 사람들인가
미세먼지 요즘 심한데같이 기재 좀 해주세요
미세먼지 요즘 심한데같이 기재 좀 해주세요
군사용으로 쓰기에는 너무 느리지 않나.....
군사용으로 쓰기에는 너무 느리지 않나
근거가 없는 공허한 외침 작렬이네
근거가 없는 공허한 외침 작렬이네
갑자기 그제부터 추워져서 대비를 못하고 있다가 약간 감기 기운이 느껴지네
갑자기 그제부터 추워져서 대비를 못하고 있다가 약간 감기 기운이 느껴지네
프랑스에 누가 갔을까 ㅋㅋㅋㅋ
프랑스에 누가 갔을까 ㅋㅋㅋㅋ
알이 얼마나 맛있는데 ㅠㅠ
알이 얼마나 맛있는데 ㅠㅠ
팩트와 증거를 제시해야지
팩트와 증거를 제시해야지
응 내일 삼성전자 주식 다 팔께.
응 내일 삼성전자 주식 다 팔께
프랑스가 이겼네 ㅎㅎㅎㅎㅎ
프랑스가 이겼네 ㅎㅎㅎㅎㅎ
요즘 피시방 가면 뭐하는데???
요즘 피시방 가면 뭐하는데
인민재판도 아니고 죄가 있어야 씌우던가 할텐데...
인민재판도 아니고 죄가 있어야 씌우던가 할텐데
스마트폰은 예전처럼 플라스틱 바디가 나은거 같기도 하네
스마트폰은 예전처럼 플라스틱 바디가 나은거 같기도 하네
잠적해서 있다가 시간맞춰서 딱 나오는 시나리오!
잠적해서 있다가 시간맞춰서 딱 나오는 시나리오
울릉도에서 출발햇습니다
울릉도에서 출발햇습니다
기대하겠습니다.
기대하겠습니다
우병우도 빨리 구속하기를~~
우병우도 빨리 구속하기를
VR 잘 선점하면 다음 조사 때는 선정 안되겠나
VR 잘 선점하면 다음 조사 때는 선정 안되겠나
왜 힘들고 어려우시다면서 계속 자리에 연연하면서 붙어 있는지 진심 궁굼하네.....
왜 힘들고 어려우시다면서 계속 자리에 연연하면서 붙어 있는지 진심 궁굼하네
홍채인식이 무슨 문제점 때문에 사용 못하게 되었는지 알면 

100%|██████████| 22532/22532 [00:33<00:00, 673.53it/s]


 친하지도 않은 여자애가 카톡하면 어떤가요
약을 너무 많은 종류를 섞어 먹여서인가여??
약을 너무 많은 종류를 섞어 먹여서인가여
아들은 비싼연봉받고 뭐한거야?
아들은 비싼연봉받고 뭐한거야
또 더워 진다고요?
또 더워 진다고요
근데 지금 6/2일 생리예정일 하루가 지났는데 생리가 안터져요..
근데 지금 62일 생리예정일 하루가 지났는데 생리가 안터져요
제가 친구들이랑 있을때는 말도 많이 하고 그런데 물론 남고라서 여자 볼 일은 거의 없는데 가끔 막 머리 자르고 그럴때 말을 거의 안하거든요..
제가 친구들이랑 있을때는 말도 많이 하고 그런데 물론 남고라서 여자 볼 일은 거의 없는데 가끔 막 머리 자르고 그럴때 말을 거의 안하거든요
 제가 먼저 말 걸어줘야될까요
 제가 먼저 말 걸어줘야될까요
 1회부터같으댓글쓰는거 진짜 없어보인다ㅋㅋ
 1회부터같으댓글쓰는거 진짜 없어보인다ㅋㅋ
진짜 태어나서 이렇게 나라를걱정한건처음.
진짜 태어나서 이렇게 나라를걱정한건처음
저 1일1식 계속진행해도 요요안오겠죠?
저 1일1식 계속진행해도 요요안오겠죠
  태연누나 이대로 가면 안티 쭉쭉 늘텐데;
  태연누나 이대로 가면 안티 쭉쭉 늘텐데
헐...아스널의 레전드를 떠올린다면 가장 먼저 언급될 정도로 아스날의 상징인 이안 라이트가 북런던 라이벌 토트넘 선수를 칭찬하는 것은 상당히 의미있는 발언이지 않나...
헐아스널의 레전드를 떠올린다면 가장 먼저 언급될 정도로 아스날의 상징인 이안 라이트가 북런던 라이벌 토트넘 선수를 칭찬하는 것은 상당히 의미있는 발언이지 않나
헌재가 만의 하나 '국민의 뜻'과 어긋난 판결을 낸다면 이 나라의 혼란은...ㅉ
헌재가 만의 하나 국민의 뜻과 어긋난 판결을 낸다면 이 나라의 혼란은ㅉ
진짜 알면 알수록 무서운 사람이다
진짜 알면 알수록 무서운 사람이다
그만해도되는거냐
그만해도되는거냐
집을 팽개쳐두고 내 앞길만 가려니 마음이 쓰인다.
집을 팽개쳐두고 내 앞길만 가려니 마음이 쓰인다
근데 남자친구가 집이 잘사는것도 아니고...
근데 남자친구가 집이 잘사는것도 아니

In [23]:
set(train_labels)

{0, 1}

In [24]:
train_inputs

(array([[   2, 3803, 6812, ...,    1,    1,    1],
        [   2, 3106, 6573, ...,    1,    1,    1],
        [   2, 1895, 5655, ...,    1,    1,    1],
        ...,
        [   2, 3343, 6812, ...,    1,    1,    1],
        [   2, 3257, 6799, ...,    1,    1,    1],
        [   2, 2363, 5561, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [25]:
def create_model(max_length=300):

    bert_base_model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True) 

    input_token_ids   = layers.Input((max_length,), dtype=tf.int32, name='input_token_ids')   # tokens_tensor
    input_masks       = layers.Input((max_length,), dtype=tf.int32, name='input_masks')       # masks_tensor
    input_segments    = layers.Input((max_length,), dtype=tf.int32, name='input_segments')    # segments_tensor  

    bert_outputs = bert_base_model([input_token_ids, input_masks, input_segments]) 
    

    bert_outputs = bert_outputs[1] # ('pooler_output', <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)
    bert_outputs = layers.Dropout(0.2)(bert_outputs)
    final_output = layers.Dense(units=2, activation='softmax', kernel_initializer=initializers.TruncatedNormal(stddev=0.02), name="classifier")(bert_outputs)

    model = tf.keras.Model(inputs=[input_token_ids, input_masks, input_segments], 
                        outputs=final_output)

    model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5, weight_decay=0.0025, warmup_proportion=0.05),
                  loss=losses.SparseCategoricalCrossentropy(), 
                  metrics=[metrics.SparseCategoricalAccuracy()])
    
    return model

In [30]:
# TPU setting

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("\nAll devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope(): 
    model = create_model(max_length=300)

KeyError: ignored

In [31]:
import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

In [32]:
#  Checkpoint setting for saving the best model
from google.colab import drive
drive.mount('/gdrive')

checkpoint_path = '/gdrive/MyDrive/final/colab_data/saved_models/'

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

callback_checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_path + 'best_bert_weights.h5', 
                                                monitor='val_sparse_categorical_accuracy',
                                                save_best_only=True, 
                                                save_weights_only = True, 
                                                verbose=1) 
                                                
# Early-stopping for preventing the overfitting
callback_earlystop = callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', 
                                             min_delta=0.0001, 
                                             patience=5)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [33]:
model = create_model(max_length=300)

In [ ]:
history = model.fit(train_inputs, train_labels, validation_split=0.2,
                    epochs=10, batch_size=200,
                    verbose=1,
                    callbacks=[callback_checkpoint, callback_earlystop])

Epoch 1/10


In [ ]:
SEQ_LEN = 300 # 최대 token 개수 이상의 값으로 임의로 설정

token_ids =[]
token_masks =[]
token_segments =[]

test_labels = []


for idx in tqdm(range(len(X_test))):

    test_sentence = X_test.iloc[idx]
    
    # 특수문자 제거
    cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", test_sentence) 
    
    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=cleaned_sentence,
                                         padding='max_length', 
                                         truncation=True,
                                         max_length=SEQ_LEN)
    
    token_ids.append(encoded_dict['input_ids']) # tokens_tensor
    token_masks.append(encoded_dict['attention_mask']) # masks_tensor
    token_segments.append(encoded_dict['token_type_ids']) # segments_tensor

    test_labels.append(y_test.iloc[idx])


test_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
test_labels = np.array(test_labels)

In [ ]:
model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5') # Load the best model's weights from checkpoint file

preds = model.predict(test_inputs)
preds = tf.argmax(preds, axis=1)

print("Accuracy: ",accuracy_score(preds, test_labels))

In [ ]:
pd.DataFrame([preds],columns=['pred'])

In [ ]:
from sklearn.metrics import precision_score , recall_score , confusion_matrix, classification_report

target_names = ["일반","혐오"] # {"일반":0, "혐오":1}

precision = precision_score(test_labels, preds,average= "macro")
recall = recall_score(test_labels, preds,average= "macro")

print("< Confusion Matrix >\n\n",confusion_matrix(test_labels, preds))
print("\n")
print("< Classification Report >\n\n",classification_report(test_labels, preds, target_names=target_names))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_test, y_test)

# roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]) 
AUC = roc_auc_score(test_labels, preds)     

# y_pred = model.predict(X_test)
precision = precision_score(test_labels, np.round(abs(preds))) 
f1 = f1_score(test_labels, np.round(abs(preds))) 

print("AUC : ", AUC)
print("Precision Score : ", precision)
print("f1 Score : ", f1)